# Structural Biology for Lead Discovery - Practical Class 1: REST interfaces in ChEMBL and Pubchem

## Synopsis

Getting programmatic access to databases allows us to go from muggle to wizard: we have better access to the data and can integrate it seamlessly into our pipelines.

In [44]:
import json  # lets us work with the json format
import requests  # allows Python to make web requests
import pandas as pd # analysis of tabular data
import rdkit.Chem as chem

## ChEMBL

### The base URL 

A REST GET query consists of using a URL address that specifies the data we want to obtain, and a `GET` header that indicates to the server the kind of operation we want to do.

In this case, we will be using:

    base_url = "https://www.ebi.ac.uk/chembl/api/data/{:s}" 

In [45]:
base_url = "https://www.ebi.ac.uk/chembl/api/data/{:s}" 

### Getting a molecule by its ID

The question, then, is how we encode our queries into an URL. The answer is by learning how the API works. Let's illustrate this by simply getting a molecule through the REST API. In this case, we will be using 

    base_url/molecule/MOLECULE_ID

**Note that when I write in upper characters, it means that is what you need to replace**.

In [46]:
chembl_id = "CHEMBL25"
molecule_url = base_url.format(f"molecule/{chembl_id}")
molecule_url

'https://www.ebi.ac.uk/chembl/api/data/molecule/CHEMBL25'

Now, we do the query. In Python, we use the requests module.

In [47]:
response = requests.get(molecule_url)

if the response went alright, we should be getting a 200 code

In [48]:
assert(response.status_code==200)
response.status_code

200

Nice, so now, let's see what we got:

In [49]:
# Print the content of the response
print("Response content:")
print(response.content)

Response content:
b"<?xml version='1.0' encoding='utf-8'?>\n<molecule><atc_classifications><level5>B01AC06</level5><level5>N02BA01</level5><level5>N02BA51</level5><level5>A01AD05</level5><level5>N02BA71</level5></atc_classifications><availability_type>2</availability_type><biotherapeutic/><black_box_warning/><chemical_probe/><chirality>2</chirality><cross_references><molecule><xref_id>aspirin</xref_id><xref_name>aspirin</xref_name><xref_src>DailyMed</xref_src></molecule></cross_references><dosed_ingredient>True</dosed_ingredient><first_approval>1950</first_approval><first_in_class/><helm_notation/><inorganic_flag/><max_phase>4.0</max_phase><molecule_chembl_id>CHEMBL25</molecule_chembl_id><molecule_hierarchy><active_chembl_id>CHEMBL25</active_chembl_id><molecule_chembl_id>CHEMBL25</molecule_chembl_id><parent_chembl_id>CHEMBL25</parent_chembl_id></molecule_hierarchy><molecule_properties><alogp>1.31</alogp><aromatic_rings>1</aromatic_rings><full_molformula>C9H8O4</full_molformula><full_mw

### Issues

- We got an XML (I think that all sensible people should hate XML files)
- Finding information here is very complicated. 

But this is not a final boss problem. We can use the `headers` to request a prettier format, such as JSON.

In [50]:
# Send a GET request to the API and retrieve the JSON response
response = requests.get(molecule_url, headers={"Accept": "application/json"})

# Check if the response is successful
assert(response.status_code == 200)
# Convert the response json into a dictionary and after that in a data frame
molecule_request = response.json()
molecule_request

{'atc_classifications': ['B01AC06',
  'N02BA01',
  'N02BA51',
  'A01AD05',
  'N02BA71'],
 'availability_type': 2,
 'biotherapeutic': None,
 'black_box_warning': 0,
 'chemical_probe': 0,
 'chirality': 2,
 'cross_references': [{'xref_id': 'aspirin',
   'xref_name': 'aspirin',
   'xref_src': 'DailyMed'}],
 'dosed_ingredient': True,
 'first_approval': 1950,
 'first_in_class': 0,
 'helm_notation': None,
 'inorganic_flag': 0,
 'max_phase': '4.0',
 'molecule_chembl_id': 'CHEMBL25',
 'molecule_hierarchy': {'active_chembl_id': 'CHEMBL25',
  'molecule_chembl_id': 'CHEMBL25',
  'parent_chembl_id': 'CHEMBL25'},
 'molecule_properties': {'alogp': '1.31',
  'aromatic_rings': 1,
  'full_molformula': 'C9H8O4',
  'full_mwt': '180.16',
  'hba': 3,
  'hbd': 1,
  'heavy_atoms': 13,
  'mw_freebase': '180.16',
  'np_likeness_score': '0.12',
  'num_ro5_violations': 0,
  'psa': '63.60',
  'qed_weighted': '0.55',
  'ro3_pass': 'N',
  'rtb': 2},
 'molecule_structures': {'canonical_smiles': 'CC(=O)Oc1ccccc1C(=O)O

This is nicer to explore: it is just a dictionary. Now, let's make it even prettier. Let's turn it into a table

In [51]:
pd.DataFrame.from_records([response.json()]).T

,0
atc_classifications,"[B01AC06, N02BA01, N02BA51, A01AD05, N02BA71]"
availability_type,2
biotherapeutic,None
black_box_warning,0
chemical_probe,0
chirality,2
cross_references,"[{'xref_id': 'aspirin', 'xref_name': 'aspirin'..."
dosed_ingredient,True
first_approval,1950
first_in_class,0


### Understanding the ChEMBL API

How did I know I had to do `/molecules/CHEMBL25`? By going to your best friend when you are dealing with APIs: the [documentation](https://chembl.gitbook.io/chembl-interface-documentation/web-services/chembl-data-web-services)

Note: This other document is also interesting, https://www.ebi.ac.uk/chembl/api/data/docs

### More advanced queries

Some of the information that we just found is something that we could have already obtained with rdkit. Now, let's go deeper: let's try to obtain a biochemical assay on a given cancer target ("CHEMBL206")

In [52]:
# ChEMBL ID of the molecule

chembl_id = "CHEMBL206"
activity_url = base_url.format (f"activity?target_chembl_id__exact={chembl_id}")
response = requests.get(activity_url, headers={"Accept": "application/json"})
assert(response.status_code == 200)
activity_request = response.json()
activity_table = pd.DataFrame.from_dict(activity_request['activities'])[['molecule_chembl_id', 'type', 'standard_value', 'standard_units']]
activity_table

,molecule_chembl_id,type,standard_value,standard_units
0,CHEMBL431611,IC50,2.5,nM
1,CHEMBL316132,IC50,7.5,nM
2,CHEMBL338926,RBA,0.84,None
3,CHEMBL338926,RBA,8.7,None
4,CHEMBL127736,RBA,75.0,None
5,CHEMBL304552,IC50,3.1,nM
6,CHEMBL85881,IC50,3.9,nM
7,CHEMBL127941,RBA,0.5,None
8,CHEMBL127941,RBA,None,None
9,CHEMBL85536,IC50,7.4,nM


We get a table with all these values. Now, let's try to sort them out by activity?

In [53]:
#Order by value
activity_table.sort_values(['standard_value'], ascending=[True])

,molecule_chembl_id,type,standard_value,standard_units
10,CHEMBL434629,RBA,0.02,None
18,CHEMBL127246,RBA,0.1,None
11,CHEMBL434629,RBA,0.15,None
16,CHEMBL124197,RBA,0.44,None
13,CHEMBL124708,RBA,0.45,None
7,CHEMBL127941,RBA,0.5,None
2,CHEMBL338926,RBA,0.84,None
15,CHEMBL267385,IC50,1.0,nM
17,CHEMBL124197,RBA,10.8,None
14,CHEMBL124708,RBA,14.8,None


**Question**: What is wrong??

### Data queries in Pandas

Pandas is not simply a pretty looking table reader and writer. It also allows to do some queries


- Sort DataFrame by a specific column in ascending order
    

    ```df.sort_values(by='column_name', ascending=True, inplace=True)```

- Sort DataFrame by multiple columns in descending order

    ```df.sort_values(by=['col1', 'col2'], ascending=[False, True], inplace=True)```

- Filter rows where a column's value meets a specific condition

    ```filtered_df = df[df['column_name'] > 100]```

- Filter rows based on multiple conditions using logical operators (AND: & , OR: |)
    
    ```filtered_df = df[(df['column1'] > 50) & (df['column2'] < 100)]```

- Keep specific columns in the DataFrame

    ```selected_columns_df = df[['column1', 'column2']]```

- Drop specific columns from the DataFrame

    ```filtered_df = df.drop(columns=['column_to_drop1', 'column_to_drop2'])```

- Filter rows with NaN values in a specific column

    ```filtered_df = df[df['column_name'].isnull()]```

- Filter rows without NaN values in a specific column
    
    ```filtered_df = df[df['column_name'].notnull()]```

### Drug mechanisms

ChEMBL has also information that we can query online to understand each drug mechanism

In [54]:
mechanism_url = base_url.format (f"mechanism?target_chembl_id__exact={chembl_id}&limit=10")
response = requests.get(mechanism_url, headers={"Accept": "application/json"})
assert(response.status_code == 200)
mechanism_request = response.json()
mechanism_request

{'mechanisms': [{'action_type': 'AGONIST',
   'binding_site_comment': None,
   'direct_interaction': 1,
   'disease_efficacy': 1,
   'max_phase': 4,
   'mec_id': 855,
   'mechanism_comment': None,
   'mechanism_of_action': 'Estrogen receptor alpha agonist',
   'mechanism_refs': [{'ref_id': '10230677',
     'ref_type': 'PubMed',
     'ref_url': 'http://europepmc.org/abstract/MED/10230677'}],
   'molecular_mechanism': 1,
   'molecule_chembl_id': 'CHEMBL1201477',
   'parent_molecule_chembl_id': 'CHEMBL1201477',
   'record_id': 1344458,
   'selectivity_comment': None,
   'site_id': None,
   'target_chembl_id': 'CHEMBL206',
   'variant_sequence': None},
  {'action_type': 'MODULATOR',
   'binding_site_comment': None,
   'direct_interaction': 1,
   'disease_efficacy': 1,
   'max_phase': 4,
   'mec_id': 856,
   'mechanism_comment': 'Mixed agonist/antagonist. Used to treat (ERα)–positive breast carcinoma and the alpha subform is the drug target.',
   'mechanism_of_action': 'Estrogen receptor al

Let's get a table

In [55]:
mechanism_table = pd.DataFrame.from_dict(mechanism_request['mechanisms'])[['molecule_chembl_id', 'target_chembl_id', 'max_phase']]
mechanism_table

,molecule_chembl_id,target_chembl_id,max_phase
0,CHEMBL1201477,CHEMBL206,4
1,CHEMBL786,CHEMBL206,4
2,CHEMBL1405,CHEMBL206,4
3,CHEMBL135,CHEMBL206,4
4,CHEMBL1200430,CHEMBL206,4
5,CHEMBL1511,CHEMBL206,4
6,CHEMBL3185958,CHEMBL206,4
7,CHEMBL691,CHEMBL206,4
8,CHEMBL1018,CHEMBL206,4
9,CHEMBL1200973,CHEMBL206,4


### Identifiers outside ChEMBL

UNIPROT and CHEMBL have different identifiers. Let's start by finding the Chembl identifier from Uniprot.

In [56]:
target_protein_url = base_url.format("target_component?accession=P03372")
target_components = requests.get(target_protein_url, headers={"Accept":"application/json"}).json()['target_components']
target_components

[{'accession': 'P03372',
  'component_id': 410,
  'component_type': 'PROTEIN',
  'description': 'Estrogen receptor',
  'go_slims': [{'go_id': 'GO:0003674'},
   {'go_id': 'GO:0003677'},
   {'go_id': 'GO:0003682'},
   {'go_id': 'GO:0005515'},
   {'go_id': 'GO:0005516'},
   {'go_id': 'GO:0005575'},
   {'go_id': 'GO:0005634'},
   {'go_id': 'GO:0005654'},
   {'go_id': 'GO:0005737'},
   {'go_id': 'GO:0005794'},
   {'go_id': 'GO:0005829'},
   {'go_id': 'GO:0005886'},
   {'go_id': 'GO:0006629'},
   {'go_id': 'GO:0007165'},
   {'go_id': 'GO:0008134'},
   {'go_id': 'GO:0008150'},
   {'go_id': 'GO:0008152'},
   {'go_id': 'GO:0008283'},
   {'go_id': 'GO:0008289'},
   {'go_id': 'GO:0016020'},
   {'go_id': 'GO:0019899'},
   {'go_id': 'GO:0030154'},
   {'go_id': 'GO:0030234'},
   {'go_id': 'GO:0040007'},
   {'go_id': 'GO:0042802'},
   {'go_id': 'GO:0043167'},
   {'go_id': 'GO:0048856'},
   {'go_id': 'GO:0065003'}],
  'organism': 'Homo sapiens',
  'protein_classifications': [{'protein_classification_i

In [57]:
targets_list = ';'.join([i['target_chembl_id'] for i in target_components[0]['targets']])
targets_url = base_url.format("target/set/{:s}".format(targets_list))
targets = requests.get(targets_url, headers={"Accept":"application/json"}).json()

In [58]:
pd.DataFrame([(item['target_chembl_id'], item['target_type']) for item in targets['targets']])

,0,1
0,CHEMBL206,SINGLE PROTEIN
1,CHEMBL2093866,PROTEIN FAMILY
2,CHEMBL3885521,PROTEIN COMPLEX
3,CHEMBL4523681,PROTEIN-PROTEIN INTERACTION
4,CHEMBL4523713,PROTEIN-PROTEIN INTERACTION
5,CHEMBL4523721,PROTEIN-PROTEIN INTERACTION
6,CHEMBL4523726,PROTEIN-PROTEIN INTERACTION
7,CHEMBL4523754,PROTEIN-PROTEIN INTERACTION


## PubChem

PubChem provides several ways for programmatic access to its data:

- PUG-REST: a simplified access route to PubChem without the overhead of XML or SOAP envelopes. PUG-REST provides convenient access to information on PubChem records not possible with other PUG services.

- PUG-View: REST-style web service that provides full reports, including third-party textual annotation, for individual PubChem records.

- Power User Gateway (PUG): PUG provides programmatic access to PubChem services via a single common gateway interface (CGI), available at http://pubchem.ncbi.nlm.nih.gov/pug/pug.cgi.

- PUG-SOAP: web service access to PubChem data using the Simple Object Access Protocol (SOAP).

- PubChemRDF REST interface: a REST-style interface designed to access RDF-encoded PubChem data.

- Entrez Utilities: E-utils are a set of programs used to access information contained in the Entrez system.

### PUG-REST

How PUG REST Works:
The URL Path: Most – if not all – of the information the service needs to produce its results is encoded into the URL. 


The conceptual framework of this service is the three-part request: 
- input – that is, what identifiers are we talking about (ID, name, SMILEs, InChl...); Identifier: SID, CID, AID

- operation – what to do with those identifiers; 

- output – what information should be returned: xml, sdf, png, txt,... 


**Design of the URL:**
PUG REST is entirely based on HTTP (or HTTPS) requests
"https://pubchem.ncbi.nlm.nih.gov/rest/pug"

   **Input**: 
   
            /compound/name/xxx   
	        /compound/cid/xxx
            cid | name | smiles | inchi | sdf | inchikey | formula | 
           
   **Operation:**
   
            /property/InChI
            /property/MolecularWeight
            /cids
              
              If no operation is specified at all, the default is to retrieve the entire record. 
              What operations are available    are dependent on the input domain. 


              <compound property> = property / [comma-separated list of property tags]
              Property tags: MolecularWeight, MolecularFormula,CanonicalSmiles,cids,XLogP... 
              check in the compound property table (https://pubchemdocs.ncbi.nlm.nih.gov/pug-rest)
              
   **Output:** /TXT
            /JSON
            /PNG
            
            <output specification> = XML | ASNT | ASNB | JSON | JSONP | SDF | CSV | PNG | TXT

In [59]:
# Get PubChem CID by name

name = "ibuprofen"
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/cids/JSON"

r = requests.get(url)
response = r.json()
if "IdentifierList" in response:
    cid = response["IdentifierList"]["CID"][0]
else:
    raise ValueError(f"Could not find matches for compound: {name}") #  used to raise exceptions or errors
print(f"PubChem CID for {name} is:\n{cid}")


PubChem CID for ibuprofen is:
3672


Let's get some molecular properties

In [60]:
# Get molecular weight for ibuprofen
cid ="3672"
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/MolecularWeight/JSON"

r = requests.get(url)
response = r.json()

if "PropertyTable" in response:
    mol_weight = response["PropertyTable"]["Properties"][0]["MolecularWeight"]
else:
    raise ValueError(f"Could not find matches for PubChem CID: {cid}")
print(f"Molecular weight for {name} is:\n{mol_weight}")


Molecular weight for ibuprofen is:
206.28


### Similarity search in PubChem

PubChem allows for two kinds of similarity search:

- **Fastsimilarity_2d**: PubChem substructure fingerprint and Tanimoto Index

- **Fastsimilarity_3d**: Similarity is evaluated with the shape-Tanimoto (ST) and color-Tanimoto (CT) scores, which quantify the similarity between their conformers in 3-D shape and functional group orientations, respectively. The ST and CT scores are calculated using the Gaussian-shape overlay method by Grant and Pickup, and implemented in the Rapid Overlay of Chemical Structures.However, because 3-D similarity search takes much longer than 2-D similarity search, it often exceeds the 30-second time limit and returns a time-out error, especially when the query molecule is big. 3-D similarity search uses a shape-Tanimoto of >=0.80 and a color-Tanimoto of >=0.50 as a similarity threshold. 


In [61]:
smiles = "C1COCC(=O)N1C2=CC=C(C=C2)N3C[C@@H](OC3=O)CNC(=O)C4=CC=C(S4)Cl"
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/fastsimilarity_2d/smiles/{smiles}/cids/txt?Threshold=99"

response= requests.get (url)
print (response.text)


9875401
6433119
11524901
25255944
25164166
25190310
56589668
56598114
68152323
123868009
143433422
144489887
145624236
145624246
154630277
159596345
11994745
25190129
25190130
25190186
25190187
25190188
25190189
25190190
25190248
25190249
25190250
25190251
25190252
25190311
25255845
25255945
25255946
49849874
133687098
156594383
169435566



In [62]:
# End of notebook — placeholder cell
# Nothing to run here; previous cells demonstrate PubChem similarity search usage.